In [30]:
import numpy as np
import pandas as pd
from datascience import *
import pgeocode
import math

In [56]:
# goldman = Table.read_table('../../just tacos and burritos.csv')
# goldman = goldman.sample(1000)
# goldman.show(15)

In [59]:
# stores = goldman.group('websites')
# stores.show(10)

In [58]:
# cuisines = goldman.group('cuisines')
# cuisines.show(5)

In [5]:
# locs = goldman.group('country')
# locs.show()

country,count
US,1000


In [6]:
goldman = goldman.drop('country').drop('id').drop('dateAdded').drop('dateUpdated').drop('menus.dateSeen').drop('menus.currency')

In [7]:
# goldman.sample(200).show()
categories = []
goldman.column('menus.name').item(9)
for i in range(0, len(goldman.column('menus.name'))):
    name = goldman.column('menus.name').item(i)
    if 'burrito' in name or 'Burrito' in name:
        categories.append('Burrito')
    else:
        categories.append('Taco')

In [8]:
goldman = goldman.with_column('Categories', categories)
for i in range(25, 30):
    name = 'Unnamed: '+str(i)
    goldman.drop(name)

In [60]:
piv = goldman.pivot('cuisines', 'name')
# piv.show(5)

In [11]:
df = goldman.to_df()

In [12]:
df2 = df.drop(['Unnamed: 235'], axis = 1)

In [13]:
for i in range(26, 236):
    name = 'Unnamed: '+str(i)
    df = df.drop([name], axis = 1)

In [14]:
df.to_csv('goldman.csv')

In [15]:
goldman2 = Table.read_table('goldman.csv')
goldman2 = goldman2.drop('Unnamed: 0').drop('menus.category').drop('priceRangeCurrency')

In [16]:
burrito = goldman2.where('Categories', are.equal_to('Burrito'))
taco = goldman2.where('Categories', are.equal_to('Taco'))

In [61]:
# burrito.show(10)
# taco.show(10)

In [62]:
burrito_rat = burrito.group('city')
# burrito_rat.show(5)

In [63]:
taco_rat = taco.group('city')
# taco_rat.show(5)

In [64]:
ratio = taco_rat.join('city', burrito_rat, 'city')
# ratio.show(10)

In [21]:
ratio.relabel('count', 'Tacos')
ratio.relabel('count_2', 'Burritos')

city,Tacos,Burritos
Albuquerque,3,4
Alpharetta,2,1
Anaheim,2,1
Atlanta,2,2
Aurora,1,1
Azusa,2,3
Bakersfield,1,2
Baltimore,3,1
Beeville,1,1
Boise,2,2


In [22]:
ratio = ratio.with_columns('Ratio of Burritos/Tacos', ratio.column('Burritos')/ratio.column('Tacos'))

## Question 1

In [66]:
ratio.show(10)

city,count,count_2
Albuquerque,3,4
Alpharetta,2,1
Anaheim,2,1
Atlanta,2,2
Aurora,1,1
Azusa,2,3
Bakersfield,1,2
Baltimore,3,1
Beeville,1,1
Boise,2,2


## #2

In [65]:
# goldman2.group('city')

In [67]:
# goldman = Table.read_table('../../just tacos and burritos.csv')
# # 

In [68]:
# goldman = goldman.sample(10000)
# id_g = goldman.group('city')
# id_g.show(10)

In [69]:
# locations = []
# count = 1
# for i in id_g.column('id'):
#     city = goldman.where('id', are.equal_to(i)).column('city').item(0)
#     print(city, count)
#     locations.append(city)
#     count += 1

In [31]:
# id_g = id_g.with_column('City', locations)

In [70]:
# id_g

In [123]:
def ratio(goldman2):
    burrito = goldman2.where('Categories', are.equal_to('Burrito'))
    taco = goldman2.where('Categories', are.equal_to('Taco'))
    
    burrito_rat = burrito.group('postalCode').relabel('count', 'burrito')
    taco_rat = taco.group('postalCode').relabel('count', 'taco')
    
    ratio = pd.merge(burrito_rat.to_df(), taco_rat.to_df(), how = "outer")
    
    ratio['Ratio of Burritos/Taco'] = ratio['burrito']/ratio['taco']
    ratio.to_csv('training.csv')
    return ratio

In [129]:
def category(goldman):
    # goldman.sample(200).show()
    categories = []
    goldman.column('menus.name').item(9)
    for i in range(0, len(goldman.column('menus.name'))):
        name = goldman.column('menus.name').item(i)
        if 'burrito' in name or 'Burrito' in name:
            categories.append('Burrito')
        else:
            categories.append('Taco')
    goldman = goldman.with_column('Categories', categories)
    df = goldman.to_df()
    
    for i in range(26, 236):
        name = 'Unnamed: '+str(i)
        df = df.drop([name], axis = 1)
    df.to_csv('part1.csv')
    return df

In [36]:
goldman = Table.read_table('../../just tacos and burritos.csv')

In [281]:
goldman = goldman.sample(10000)
f = category(goldman)

g = Table.read_table('part1.csv')


In [279]:
goldman2 = Table.read_table('goldman.csv')
goldman2 = goldman2.drop('Unnamed: 0').drop('menus.category').drop('priceRangeCurrency')
r = ratio(goldman2)

In [282]:
g = Table().with_columns('Name', g.column('name'), 'postalCode', g.column('postalCode'), 'categories', g.column('categories') )
g.show(10)

Name,postalCode,categories
Chili's Grill & Bar,77584,Tex-Mex Restaurant
Tortilla Jo's,92802,"Restaurants,Mexican Restaurant,Latin American Restaurant ..."
Blue Parrot,80027,Italian Restaurant
Java Joe's,87102,"Coffee Shop,Restaurants,Coffeehouses"
El Pollo Loco,92505,"Fast Food Restaurant,Mexican Restaurant"
Cantina Feliz,19034,"Mexican Restaurant,Bar"
Pocos Restaurant,91303,"Mexican Restaurant Canoga Park,Mexican Restaurant"
Cactus Restaurant,98112,"Latin American Restaurants,Take Out Restaurants,Caterers ..."
Mcdonald's,97739,Fast Food Restaurant
Chili's Grill & Bar,77584,Tex-Mex Restaurant


In [259]:
g = g.group('postalCode', max)
g

postalCode,Name max
10065,Maya - New York
11201,The Original Fresco Tortillas Plus
11237,Taqueria El Paisa
11559,Carlos & Gabby's
12866,The Brook Tavern
15931,Mcdonald's
1610,Peppercorn's Grille & Bar
16830,McDonald's
17033,Minetown Restaurant
18034,Torre Restaurant


In [ ]:
model_table = model_table.to_df()
model_table = pd.merge(model_table, taco)

In [80]:
model_table.to_csv('training.csv')

In [67]:
def longlat(goldman):
    longs = []
    lats = []
    nomi = pgeocode.Nominatim('fr')
    for i in range(0, len(goldman.column('latitude'))):
        if math.isnan(goldman.column('latitude').item(i)):
            zipcode = goldman.column('postalCode').item(i)
            lon = nomi.query_postal_code(str(zipcode))['longitude']
            lat = nomi.query_postal_code(str(zipcode))['latitude']
            longs.append(lon)
            lats.append(lat)
            print(lon, lat, goldman.column('city').item(i), zipcode)
        else:   
            longs.append(goldman.column('longitude').item(i))
            lats.append(goldman.column('latitude').item(i))
            
    return [longs,lats]

In [167]:
zips = Table.read_table('zipcodes.csv')


In [225]:
lala = Table.read_table('r.csv')
lala = lala.drop('Unnamed: 0')
# lala.column('postalCode')

In [166]:
zipzz = Table().with_columns('postalCode', zips.column(0),
                            'Longitude', zips.column(1),
                            'Latitude', zips.column(2))
post = [str(i) for i in zipzz.column('postalCode')]
zipzz = zipzz.drop('postalCode').with_column('postalCode', post)
zipzz.column('postalCode')

array(['501', '544', '601', ..., '99928', '99929', '99950'], dtype='<U5')

In [211]:
final = zipzz.join('postalCode', lala, 'postalCode') 
final = final.join('postalCode', g, 'postalCode')
final

postalCode,Longitude,Latitude,burrito,taco,Ratio of Burritos/Taco,Name max
10065,-73.9638,40.7653,nan,67,nan,Maya - New York
11201,-73.9927,40.6955,18,nan,nan,The Original Fresco Tortillas Plus
11237,-73.9216,40.7046,nan,44,nan,Taqueria El Paisa
11559,-73.7323,40.6118,5,nan,nan,Carlos & Gabby's
12866,-73.7748,43.076,30,6,5,The Brook Tavern
15931,-78.7326,40.4838,60,nan,nan,Mcdonald's
1610,-71.808,42.249,nan,28,nan,Peppercorn's Grille & Bar
16830,-78.4416,41.0231,12,nan,nan,McDonald's
17033,-76.6307,40.2788,nan,55,nan,Minetown Restaurant
18034,-75.4256,40.5483,29,nan,nan,Torre Restaurant


In [283]:
final.to_csv('formap.csv')
final = final.to_df()
# final

AttributeError: 'DataFrame' object has no attribute 'to_df'

In [213]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [214]:
# Elbow method to determine the number of K in Kmeans Clustering
coords = final[['Longitude', 'Latitude']]

distortions = []
K = range(1,25)
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    distortions.append(kmeansModel.inertia_)

In [215]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(K, distortions, marker='o')
plt.xlabel('k')
plt.ylabel('Distortions')
plt.title('Elbow Method For Optimal k')
plt.savefig('elbow.png')
plt.show()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [216]:
from sklearn.metrics import silhouette_score

sil = []
kmax = 50

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(coords)
  labels = kmeans.labels_
  sil.append(silhouette_score(coords, labels, metric = 'euclidean'))

In [217]:
kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(coords)
y = kmeans.labels_
print("k = 5", " silhouette_score ", silhouette_score(coords, y, metric='euclidean'))

k = 5  silhouette_score  0.5162431460875556


In [218]:
final['cluster'] = kmeans.predict(final[['Longitude','Latitude']])


In [223]:
def recommend_restaurants(df, longitude, latitude):
    # Predict the cluster for longitude and latitude provided
    cluster = kmeans.predict(np.array([longitude,latitude]).reshape(1,-1))[0]
    print(cluster)
   
    # Get the best restaurant in this cluster
    return  df[df['cluster']==cluster].iloc[0:5][['postalCode', 'Latitude','Longitude']]

In [241]:
tab = recommend_restaurants(final,-115.2798544, 36.0842838)
tab.to_csv('tab.csv')

1


In [266]:
tab1 = Table.read_table('tab.csv')
tab1 = tab1.drop('Unnamed: 0')

In [267]:
g = g.drop('Unnamed: 0')
g.show(4)
tab1.show(4)

postalCode,Name max
10065,Maya - New York
11201,The Original Fresco Tortillas Plus
11237,Taqueria El Paisa
11559,Carlos & Gabby's


postalCode,Latitude,Longitude
83501,46.389,-116.986
83642,43.6091,-116.398
83706,43.5905,-116.19
89030,36.211,-115.129


In [285]:
import matplotlib

In [286]:
import numpy

In [287]:
import pandas

In [291]:
from matplotlib.pyplot import figure
import mpld3

fig = figure()
ax = fig.gca()
ax.plot([1,2,3,4])

mpld3.save_html(fig,'myfig.html')
#mpld3.show(fig)
